# Dicom file manipulation with the Pydicom Library
https://pydicom.github.io/

## Contents:
* Reading Dicom files
* Modifying Dicom images
* Changing Dicom tags
* Reading Dicom CT image files
* Reading Dicom RT Plan files

###  Architecture of DICOM :
### https://dicom.innolitics.com/ciods

# Reading Dicom files (general)

## Exemple with a Nuclear Medicine imaging (SPECT) Dicom

In [ ]:
import pydicom as dcm

# open dicom file (works for all .dcm file)
File=dcm.read_file('data/patient_SPECT.dcm')
print(File)

## Acces to Dicom data by sequence name 

In [ ]:
print('Image type :',  File.ImageType )
print('Image type first element :', File.ImageType[0])
print('Modality: ',  File.Modality)
print()
# Type of datas
# Note that they are all string, so you need a conversion to (float, int...)
print('type of ImageType dicom sequence : ', type(File.ImageType))
print('type of first element of ImageType dicom sequence : ', type(File.ImageType[0]))
print()
pixelSpacing = File.PixelSpacing
print ('pixel spacing :', pixelSpacing)
print ('length pixel spacing sequence :', len(pixelSpacing))
print ('type pixel spacing :', type(pixelSpacing[0]))

In [ ]:
# Search dicom sequence with 'name' chain
print (File.dir('name'))

## Acces to Dicom data by tag (hexadecimal)

In [ ]:
#print 'Detector Information Sequence' dicom sequence 
print (File[0x054, 0x022])

#print 'Radialposition' First sub sequence of 'Detector Information Sequence' dicom sequence
print (File[0x054, 0x022][0][0x018, 0x1142].value)

In [ ]:
list_0 = File[0x054, 0x022][0][0x018, 0x1142].value
list_1 = File[0x054, 0x022][1][0x018, 0x1142].value
# Note the loop to convert string into float into the 'list' pos_radial_0
pos_radial_0 = [float(val) for val in list_0]
pos_radial_1 = [float(val) for val in list_1]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# plot the positions
plt.plot(pos_radial_1+pos_radial_0, 'o-', label='detecteur 1+2');

In [ ]:
import numpy as np

# this polar plot with the same values describes camera trajectory surronding patient
plt.polar([2*np.pi*l/64 for l in range(64)],pos_radial_1+pos_radial_0);

## Acces to Image pixel map

In [ ]:
# Acces to image pixel data 
arr_image = File.pixel_array
print (type(arr_image))
print ('Image Dimension : ', File.pixel_array.shape)
print(File.pixel_array.min(), File.pixel_array.max(), File.pixel_array.mean())

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

def im_viewer(im,cmap):
    def plot_image(myslice):
        plt.imshow(im[myslice,:,:], cmap=cmap)
    interact(plot_image, myslice=(0, im.shape[0]-1))
    return;

im_viewer(arr_image, plt.cm.hot)

## Creation of a dictionary on the  Energy Windows (there are 5)

In [ ]:
# Looking for dicom sequence
File.dir('energy')

In [ ]:
print (File.EnergyWindowInformationSequence)
print 
print ('there are', len(File.EnergyWindowInformationSequence), 'energies')

In [ ]:
# Example of two way to acces to Sequence Data
print (File.EnergyWindowInformationSequence[0].EnergyWindowRangeSequence[0].EnergyWindowLowerLimit)
print (File.EnergyWindowInformationSequence[0][0x054, 0x013][0][0x054, 0x014].value)

In [ ]:
#Dictionary creation 
dic_energy={}

# Loop on every energy of EnergyWindowInformationSequence and collect data 
for energy in File.EnergyWindowInformationSequence:
    low=float(energy[0x054, 0x013][0][0x054, 0x014].value)
    high=float(energy[0x054, 0x013][0][0x054, 0x015].value)
    width=high-low
    pic=(high+low)/2
    pic_name=round((high+low)/2, 0)        #round to get int part 
    dic_energy[pic_name]={}
    dic_energy[pic_name]['low']=low
    dic_energy[pic_name]['high']=high
    dic_energy[pic_name]['width']=width
    dic_energy[pic_name]['pic']=pic

dic_energy

# Modifying Dicom files

## Changing the image data

In [ ]:
# modify and save
print (File.pixel_array.min(), File.pixel_array.max(), File.pixel_array.mean())
File.pixel_array[File.pixel_array < 100] = 0
print (File.pixel_array.min(), File.pixel_array.max(), File.pixel_array.mean())

# the pixel values are contained in PixelData, pixel_array is an attribute from pydicom
# to store the new pixel values they need to be set as a string to PixelData
File.PixelData = File.pixel_array.tostring()

## Changing Dicom tags

In [ ]:

# Read Dicom file

# Access Dicom tags

# Remove patient name

## Saving Dicom file

In [ ]:
File.save_as('output/test.dcm')

# Reading Dicom CT image

In [5]:
import numpy as np
import pydicom as dcm
#from glob import glob
import os

def read_CT(path):
    for s in os.listdir(path):
        print(s)
    slices = [dcm.read_file(path + '/' + s) for s in os.listdir(path)]
    #slices = []
    #for file in glob(path + '/*.dcm'):
    #    dc = dcm.read_file(file)
    #    slices.append(dc)
    # sort the slices
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    # create a numpy matrix containing all slices
    ct = np.stack([s.pixel_array for s in slices], axis=-1)
    # Convert to Hounsfield units (HU)
    ct = ct * slices[0].RescaleSlope + slices[0].RescaleIntercept
    return ct

In [6]:
# subset of anonymized CT dicom sample from https://www.dicomlibrary.com
ct_image = read_CT('data/patient_CT_dcm/')
ct_image.shape

image-000000.dcm
image-000014.dcm
image-000015.dcm
image-000001.dcm
image-000017.dcm
image-000003.dcm
image-000002.dcm
image-000016.dcm
image-000012.dcm
image-000006.dcm
.DS_Store
image-000007.dcm
image-000013.dcm
image-000005.dcm
image-000011.dcm
image-000010.dcm
image-000004.dcm
image-000009.dcm
image-000021.dcm
image-000020.dcm
image-000008.dcm
image-000022.dcm
image-000023.dcm
image-000024.dcm
image-000018.dcm
image-000019.dcm
image-000025.dcm


InvalidDicomError: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline

def im_viewer(im,cmap):
    def plot_image(myslice):
        plt.imshow(im[:,:,myslice], cmap=cmap)
    interact(plot_image, myslice=(0, im.shape[2]-1))
    return;

im_viewer(ct_image, plt.cm.bone)

# Reading DICOM RT plan

#### DICOM input for GATE simulations:
* Beam level
    * Beam Delivery Type
    * Beam Type
    * Beam Radiation Type
    * Beam Energy
    * Beam Fluence Type
    * Direction Gantry Rotation
    * Direction Collimator Rotation
    * Collimator Rotation
    * Isocenter Position
    * Applicator ID (electron beam)
    * Applicator Type (electron beam)
    * Source Surface Distance (electron beam)
* Control Point Index
    * Gantry Angle
    * X Jaws Poistions
    * Y Jaws Positions
    * MLC Positions
    * Dose Rate
    
    


In [ ]:
import pydicom as dcm

plan = dcm.read_file("data/patient_RP.dcm")
print(plan)

In [ ]:
for beam in plan.BeamSequence:
    print('Beam Name: ', beam.BeamName)
    print('Beam Delivery Type: ', beam.BeamType)
    print('Beam Radiation Type: ', beam.RadiationType)
    print('Beam Energy: ', beam.ControlPointSequence[0].NominalBeamEnergy)
    print('Beam Fluence Mode: ', beam.PrimaryFluenceModeSequence[0].FluenceMode)
    
    for control_index in beam.ControlPointSequence:
        print('Control Index: ', control_index.ControlPointIndex)
        print('     *Gantry Angle: ', control_index.GantryAngle)
        print('     *Dose Rate: ', control_index.ReferencedDoseReferenceSequence[0].CumulativeDoseReferenceCoefficient)
        print('     *MLC: ', control_index.BeamLimitingDevicePositionSequence[0].LeafJawPositions) # bug to fix control index 0 MLC vlaues seem wrong

## Buid a dictionary with beam parameters

In [ ]:
dic_RP = {}
dic_RP['Name'] = plan.PatientName
dic_RP['ID'] = plan.PatientID
dic_RP['Beam'] = {}

for beam in plan.BeamSequence:
    if beam.TreatmentDeliveryType == 'TREATMENT':
        beam_name=beam.BeamName
        dic_RP['Beam'][beam_name] = {}
        dic_RP['Beam'][beam_name]['Type'] = beam.BeamType
        dic_RP['Beam'][beam_name]['RadiationType'] = beam.RadiationType
        dic_RP['Beam'][beam_name]['Energy'] = beam.ControlPointSequence[0].NominalBeamEnergy
        dic_RP['Beam'][beam_name]['ControlPointSequence'] = {}
        dic_RP['Beam'][beam_name]['ControlPointSequence']['GantryAngle'] = round(float(beam.ControlPointSequence[0].GantryAngle), 3)
        dic_RP['Beam'][beam_name]['ControlPointSequence']['DoseRate'] = round(float(beam.ControlPointSequence[0].ReferencedDoseReferenceSequence[0].CumulativeDoseReferenceCoefficient), 3)
        dic_RP['Beam'][beam_name]['ControlPointSequence']['X jaws'] = [float(val) for val in beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[0].LeafJawPositions]
        dic_RP['Beam'][beam_name]['ControlPointSequence']['Y jaws'] = [float(val) for val in beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[1].LeafJawPositions]
        dic_RP['Beam'][beam_name]['ControlPointSequence']['MLC'] = [float(val) for val in beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[2].LeafJawPositions]
          
dic_RP